In [56]:
import sqlite3
import pandas as pd
import csv
import argparse

In [57]:
db = sqlite3.connect("Covid.db")
cur = db.cursor()
con = sqlite3.connect("Covid.db")


In [58]:
df = pd.read_csv(r"/Users/jenny.wade@schibsted.com/Documents/GitHub/python_programmering/Inlämning/covid_vaccinations.csv")
df = df.fillna(0)
db = df.to_sql("Covid_vaccinations", con, schema=None, if_exists="replace", index=True, index_label=None, chunksize=None, dtype=None, method=None)


In [21]:
#Skapa dataframes från queries i en linje kod
df = pd.read_sql_query("SELECT * FROM Covid_vaccinations", con)
print(df.head())

   index  Unnamed: 0      country iso_code        date  total_vaccinations  \
0      0           0  Afghanistan      AFG  2021-02-22                 0.0   
1      1           1  Afghanistan      AFG  2021-02-23                 0.0   
2      2           2  Afghanistan      AFG  2021-02-24                 0.0   
3      3           3  Afghanistan      AFG  2021-02-25                 0.0   
4      4           4  Afghanistan      AFG  2021-02-26                 0.0   

   people_vaccinated  people_fully_vaccinated  daily_vaccinations  \
0                0.0                      0.0                 0.0   
1                0.0                      0.0              1367.0   
2                0.0                      0.0              1367.0   
3                0.0                      0.0              1367.0   
4                0.0                      0.0              1367.0   

   total_vaccinations_per_hundred  ...  Covaxin  CanSino  Abdala  \
0                             0.0  ...        0 

In [ ]:
#Creating all tables (empty) I need for my database below:

In [59]:
cur.execute("""CREATE TABLE Daily_Vaccine_Data(
            iso_code text,
            date text,
            country text,
            daily_vaccinations integer,
            daily_vaccinations_per_million,
            PRIMARY KEY(iso_code, date)
            FOREIGN KEY(country) REFERENCES Country_Source(country))""")

In [60]:
cur.execute("""CREATE TABLE Country_Source(
            country text PRIMARY KEY, 
            source_name text, 
            source_website text)""")

In [61]:
cur.execute("""CREATE TABLE Country_Vaccines(
    country text PRIMARY KEY, Johnson_and_Johnson integer, 
    Oxford_AstraZeneca integer, Sinovac integer,
    Sputnik_V integer, Moderna integer, 
    Sinopharm_Beijing integer, Covaxin integer, 
    CanSino integer, Abdala integer, 
    Pfizer_BioNTech integer, QazVac integer, 
    EpiVacCorona integer, Soberana02 integer, 
    Sinopharm_HayatVax integer, RBDDimer integer, 
    Sinopharm_Wuhan integer,
    FOREIGN KEY(country) REFERENCES Country_Source(country))""")

In [62]:
cur.execute("""CREATE TABLE Country_Total(
    iso_code text,
    date text,
    total_vaccinations integer, 
    country text , 
    people_vaccinated integer, 
    people_fully_vaccinated integer,
    total_vaccinations_per_hundred real, 
    people_vaccinated_per_hundred real, 
    people_fully_vaccinated_per_hundred real,
    PRIMARY KEY (iso_code, date)
    FOREIGN KEY(country) REFERENCES Country_Source(country))""")

In [ ]:
#Creating copy tables with all the data I need for each table already ordered

In [63]:
cur.execute("""CREATE TABLE Daily_Vaccine_Data_data AS SELECT
            iso_code, date, country, daily_vaccinations, daily_vaccinations_per_million
            FROM Covid_vaccinations""")


In [64]:
cur.execute("""CREATE TABLE Country_Source_data
            AS SELECT DISTINCT country, source_name, source_website 
            FROM Covid_vaccinations""")


In [65]:
cur.execute("""CREATE TABLE Country_vaccines_data AS SELECT DISTINCT country, 
            Johnson_and_Johnson, Oxford_AstraZeneca, Sinovac, Sputnik_V, Moderna, 
            Sinopharm_Beijing, Covaxin, CanSino, Abdala, Pfizer_BioNTech, QazVac, 
            EpiVacCorona, Soberana02, Sinopharm_HayatVax, RBD_Dimer, Sinopharm_Wuhan 
            FROM Covid_vaccinations""")


In [66]:
cur.execute("""CREATE TABLE Country_Total_data AS SELECT DISTINCT iso_code, date, country, total_vaccinations, people_vaccinated,
            people_fully_vaccinated, total_vaccinations_per_hundred, people_vaccinated_per_hundred,
            people_fully_vaccinated_per_hundred FROM Covid_vaccinations""")


In [67]:
#Insert data table into actual table
cur.execute("""INSERT INTO Country_Source SELECT * FROM Country_Source_data""")


In [68]:
#Inserting data from remaining tables into corresponding table
cur.execute("""INSERT INTO Daily_Vaccine_Data SELECT * FROM Daily_Vaccine_Data_data""")
cur.execute("""INSERT INTO Country_Vaccines SELECT * FROM Country_vaccines_data""")
cur.execute("""INSERT INTO Country_Total SELECT * FROM Country_Total_data """)


In [69]:
#After checking that all tables have the right data I'll remove the data tables along with the original
cur.execute("""DROP TABLE Daily_Vaccine_Data_data""")
cur.execute("""DROP TABLE Country_Source_data""")
cur.execute("""DROP TABLE Country_vaccines_data""")
cur.execute("DROP TABLE Country_total_data")
cur.execute("DROP TABLE Covid_vaccinations")


In [70]:
con.commit()